<a href="https://colab.research.google.com/github/S498/CMPE297Section1FinalProject/blob/main/craigslistads4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Calculated and updated the Rouge scores in df1

In [ ]:
from bs4 import BeautifulSoup #web scraping

import matplotlib.pyplot as plt #plotting
import pandas as pd
import numpy as np
import re
import requests #http requests
from requests import get
from urllib.request import urlopen

In [ ]:
pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install happytransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 2.3 MB/s 
     |████████████████████████████████| 451 kB 8.8 MB/s 
     |████████████████████████████████| 5.8 MB 59.4 MB/s 
     |████████████████████████████████| 1.3 MB 57.4 MB/s 
     |████████████████████████████████| 182 kB 83.8 MB/s 
     |████████████████████████████████| 132 kB 73.4 MB/s 
     |████████████████████████████████| 212 kB 72.9 MB/s 
     |████████████████████████████████| 127 kB 56.5 MB/s 
     |████████████████████████████████| 7.6 MB 27.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from rouge import Rouge
ROUGE = Rouge()

In [ ]:
from happytransformer import  HappyTextToText

In [ ]:
df1 = pd.read_csv('df1.csv') 
df2 = pd.read_csv('df2.csv') 

In [ ]:
happy_tt = HappyTextToText("DISTILBART", "sshleifer/distilbart-cnn-12-6")

Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
def get_tt_result(df):
  tt_result = []
  for _, item in df.iterrows():
    tt_result.append(happy_tt.generate_text(str(item.Description[:1024])).text)
  df['tt_result'] = tt_result

In [ ]:
from happytransformer.happy_text_to_text import TTSettings
top_k_sampling_settings = TTSettings(do_sample=True, top_k=100, temperature=0.7, max_length=100)

In [ ]:
def get_fine_tt_result(df):
  temp_result = []
  for _, item in df.iterrows():
    temp_result.append(happy_tt.generate_text(str(item.Description[:1024]), args= top_k_sampling_settings).text)
  df['fine_tt_result'] = temp_result

In [ ]:
def calc_rouge_scores(df, column1, column2, newColumnName):
  temp = []
  for _, item in df.iterrows():
    temp.append(ROUGE.get_scores(item[column1], item[column2], avg = True))
  df[newColumnName] = temp

In [ ]:
def sum_rpf(df, columnName):
  rouge_1_r, rouge_1_p, rouge_1_f, rouge_2_r, rouge_2_p, rouge_2_f, rouge_l_r, rouge_l_p, rouge_l_f = 0, 0, 0, 0, 0, 0, 0, 0, 0
  for _, item in df.iterrows():
    rouge_1_r += item[columnName]['rouge-1']['r']
    rouge_1_p += item[columnName]['rouge-1']['p']
    rouge_1_f += item[columnName]['rouge-1']['f']

    rouge_2_r += item[columnName]['rouge-2']['r']
    rouge_2_p += item[columnName]['rouge-2']['p']
    rouge_2_f += item[columnName]['rouge-2']['f']

    rouge_l_r += item[columnName]['rouge-l']['r']
    rouge_l_p += item[columnName]['rouge-l']['p']
    rouge_l_f += item[columnName]['rouge-l']['f']

  return pd.DataFrame.from_dict({
      'rouge-1': {
        'r': rouge_1_r/len(df),
        'p': rouge_1_p/len(df),
        'f': rouge_1_f/len(df)
      },
      'rouge-2': {
        'r': rouge_2_r/len(df),
        'p': rouge_2_p/len(df),
        'f': rouge_2_f/len(df)
      },
      'rouge-l': {
        'r': rouge_l_r/len(df),
        'p': rouge_l_p/len(df),
        'f': rouge_l_f/len(df)
      }
  })

In [ ]:
# T5
happy_t5 = HappyTextToText("T5", "t5-base")

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
def get_t5_result(df):
  t5_result = []
  for _, item in df.iterrows():
    t5_result.append(happy_t5.generate_text('summarize: '+str(item.Description[:1024])).text)
  df['t5_result'] = t5_result

In [ ]:
%%time
get_tt_result(df1)

CPU times: user 3min 3s, sys: 2.63 s, total: 3min 6s
Wall time: 3min 6s


In [ ]:
%%time
get_fine_tt_result(df1)

CPU times: user 4min 10s, sys: 3.92 s, total: 4min 13s
Wall time: 4min 23s


In [ ]:
calc_rouge_scores(df1, 'Description', 'tt_result','desc_tt_result_rouge')
calc_rouge_scores(df1, 'Description', 'fine_tt_result','desc_fine_tt_result_rouge')
df1.head()

,Unnamed: 0,Price,Location,Title,Link,Date,Description,tt_result,fine_tt_result,desc_tt_result_rouge,desc_fine_tt_result_rouge
0,0,"$3,000",Cary,2008 Honda Civic,https://raleigh.craigslist.org/cto/d/cary-2008...,Nov 24,This is Amy (my daughter names all of our cars...,Amy has 197K miles but runs like a top and st...,Amy has 197K miles but runs like a top and st...,"{'rouge-1': {'r': 1.0, 'p': 0.2302158273381295...","{'rouge-1': {'r': 1.0, 'p': 0.2877697841726618..."
1,1,"$5,900","Kenly NC,",NC 2005 Buick (low miles),https://raleigh.craigslist.org/ctd/d/kenly-nc-...,Nov 24,"Nice NC clean title 2005 Buick Lacrosse CXL, j...",2005 Buick Lacrosse CXL has a V-6 3800 engine...,"2005 Buick Lacrosse CXL, just serviced and in...","{'rouge-1': {'r': 0.8421052631578947, 'p': 0.5...","{'rouge-1': {'r': 0.8846153846153846, 'p': 0.7..."
2,2,"$5,300",Durham,Toyota Avalon Limited 2007,https://raleigh.craigslist.org/cto/d/durham-to...,Nov 24,"I want to sell my car, Toyota Avalon 2007, it ...",The Toyota Avalon 2007 is a front-wheel drive...,"The Toyota Avalon 2007 has no issues, has a g...","{'rouge-1': {'r': 0.9565217391304348, 'p': 0.2...","{'rouge-1': {'r': 0.8260869565217391, 'p': 0.4..."
3,3,"$8,500","Smithfield, N.C.",2010 MAZDA CX-9 TOURING❤️~~ JZ CARS INC ~~⭐,https://raleigh.craigslist.org/ctd/d/pine-leve...,Nov 24,"DLR#:80341\n\nJZ CARS, INC.\n4187 US HWY 70 E....",The 2010 MAZDA CX-9 is a 2010 model that has ...,The 2010 MAZDA CX-9 is a 2010 SUV and 2010 mo...,"{'rouge-1': {'r': 0.3333333333333333, 'p': 0.0...","{'rouge-1': {'r': 0.3333333333333333, 'p': 0.0..."
4,4,"$8,500","Smithfield, N.C.",2010 CHRYSLER SEBRING LIMITED CONVERTIBLE LOW ...,https://raleigh.craigslist.org/ctd/d/pine-leve...,Nov 24,"DLR#:80341\n\nJZ CARS, INC.\n4187 US HWY 70 E....",The 2010 CHRYSLER SEBRING has a 2010 model wi...,"The 2010 CHRYSLER SEBRING has $8,500 in cash ...","{'rouge-1': {'r': 0.3333333333333333, 'p': 0.0...","{'rouge-1': {'r': 0.29411764705882354, 'p': 0...."


In [ ]:
%%time
get_t5_result(df1)

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


CPU times: user 6min 2s, sys: 2.55 s, total: 6min 5s
Wall time: 6min 5s


In [ ]:
calc_rouge_scores(df1, 'fine_tt_result','t5_result','fine_tt_result_t5_result_rouge')
calc_rouge_scores(df1, 'Description','t5_result','desc_t5_result_rouge')
df1.head()

,Unnamed: 0,Price,Location,Title,Link,Date,Description,tt_result,fine_tt_result,desc_tt_result_rouge,desc_fine_tt_result_rouge,t5_result,fine_tt_result_t5_result_rouge,desc_t5_result_rouge
0,0,"$3,000",Cary,2008 Honda Civic,https://raleigh.craigslist.org/cto/d/cary-2008...,Nov 24,This is Amy (my daughter names all of our cars...,Amy has 197K miles but runs like a top and st...,Amy has 197K miles but runs like a top and st...,"{'rouge-1': {'r': 1.0, 'p': 0.2302158273381295...","{'rouge-1': {'r': 1.0, 'p': 0.2877697841726618...",i bought Amy in 2010 when my oldest child got ...,"{'rouge-1': {'r': 0.6857142857142857, 'p': 0.6...","{'rouge-1': {'r': 0.9428571428571428, 'p': 0.2..."
1,1,"$5,900","Kenly NC,",NC 2005 Buick (low miles),https://raleigh.craigslist.org/ctd/d/kenly-nc-...,Nov 24,"Nice NC clean title 2005 Buick Lacrosse CXL, j...",2005 Buick Lacrosse CXL has a V-6 3800 engine...,"2005 Buick Lacrosse CXL, just serviced and in...","{'rouge-1': {'r': 0.8421052631578947, 'p': 0.5...","{'rouge-1': {'r': 0.8846153846153846, 'p': 0.7...","2005 Buick Lacrosse CXL, just serviced and ins...","{'rouge-1': {'r': 0.8695652173913043, 'p': 0.7...","{'rouge-1': {'r': 0.9565217391304348, 'p': 0.7..."
2,2,"$5,300",Durham,Toyota Avalon Limited 2007,https://raleigh.craigslist.org/cto/d/durham-to...,Nov 24,"I want to sell my car, Toyota Avalon 2007, it ...",The Toyota Avalon 2007 is a front-wheel drive...,"The Toyota Avalon 2007 has no issues, has a g...","{'rouge-1': {'r': 0.9565217391304348, 'p': 0.2...","{'rouge-1': {'r': 0.8260869565217391, 'p': 0.4...","i want to sell my car, Toyota Avalon 2007, it ...","{'rouge-1': {'r': 0.7096774193548387, 'p': 0.4...","{'rouge-1': {'r': 0.9354838709677419, 'p': 0.3..."
3,3,"$8,500","Smithfield, N.C.",2010 MAZDA CX-9 TOURING❤️~~ JZ CARS INC ~~⭐,https://raleigh.craigslist.org/ctd/d/pine-leve...,Nov 24,"DLR#:80341\n\nJZ CARS, INC.\n4187 US HWY 70 E....",The 2010 MAZDA CX-9 is a 2010 model that has ...,The 2010 MAZDA CX-9 is a 2010 SUV and 2010 mo...,"{'rouge-1': {'r': 0.3333333333333333, 'p': 0.0...","{'rouge-1': {'r': 0.3333333333333333, 'p': 0.0...","We Also Do BUY HERE PAY HERE $4,250 Down (Taxe...","{'rouge-1': {'r': 0.08695652173913043, 'p': 0....","{'rouge-1': {'r': 0.8260869565217391, 'p': 0.2..."
4,4,"$8,500","Smithfield, N.C.",2010 CHRYSLER SEBRING LIMITED CONVERTIBLE LOW ...,https://raleigh.craigslist.org/ctd/d/pine-leve...,Nov 24,"DLR#:80341\n\nJZ CARS, INC.\n4187 US HWY 70 E....",The 2010 CHRYSLER SEBRING has a 2010 model wi...,"The 2010 CHRYSLER SEBRING has $8,500 in cash ...","{'rouge-1': {'r': 0.3333333333333333, 'p': 0.0...","{'rouge-1': {'r': 0.29411764705882354, 'p': 0....","We Also Do BUY HERE PAY HERE $4,250 Down (Taxe...","{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'r...","{'rouge-1': {'r': 0.8260869565217391, 'p': 0.2..."


In [ ]:
df1_avg_desc_tt_result_rouge = sum_rpf(df1,'desc_tt_result_rouge')
df1_avg_desc_fine_tt_result_rouge = sum_rpf(df1,'desc_fine_tt_result_rouge')
df1_avg_desc_t5_result_rouge = sum_rpf(df1,'desc_t5_result_rouge')
df1_avg_fine_tt_result_t5_result_rouge = sum_rpf(df1,'fine_tt_result_t5_result_rouge')

In [ ]:
print('df1_avg_desc_tt_result_rouge \n\n',df1_avg_desc_tt_result_rouge)
print('\ndf1_avg_desc_fine_tt_result_rouge \n\n',df1_avg_desc_fine_tt_result_rouge)
print('\ndf1_avg_desc_t5_result_rouge \n\n',df1_avg_desc_t5_result_rouge)
print('\ndf1_avg_fine_tt_result_t5_result_rouge \n\n',df1_avg_fine_tt_result_t5_result_rouge)

df1_avg_desc_tt_result_rouge 

     rouge-1   rouge-2   rouge-l
r  0.833824  0.678088  0.829514
p  0.281056  0.235096  0.279894
f  0.374923  0.304921  0.373203

df1_avg_desc_fine_tt_result_rouge 

     rouge-1   rouge-2   rouge-l
r  0.739313  0.561626  0.729449
p  0.279364  0.219641  0.275793
f  0.363455  0.276205  0.358765

df1_avg_desc_t5_result_rouge 

     rouge-1   rouge-2   rouge-l
r  0.838299  0.702175  0.832013
p  0.293415  0.245315  0.290644
f  0.388536  0.319541  0.385247

df1_avg_fine_tt_result_t5_result_rouge 

     rouge-1   rouge-2   rouge-l
r  0.430997  0.315174  0.424998
p  0.384899  0.267475  0.379449
f  0.387372  0.274020  0.381856


In [ ]:
%%time
get_tt_result(df2)
get_fine_tt_result(df2)

df2.head()

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


CPU times: user 14.1 s, sys: 175 ms, total: 14.2 s
Wall time: 14.9 s


,Unnamed: 0,Price,Location,Title,Link,Date,Description,tt_result,fine_tt_result
0,0,"$19,990",TOUCHLESS DELIVERY TO YOUR HOME,2010 Toyota RAV4 Limited Sport Utility 4D suv ...,https://boone.craigslist.org/ctd/d/boone-2010-...,Nov 23,"At Carvana, we go miles beyond the extra mile....","Carvana provides a convenient, fast, and hass...",Carvana is a leading manufacturer that helps ...
1,1,"$1,500",Newland NC,2003 ford e150 work van,https://boone.craigslist.org/cto/d/montezuma-2...,Nov 22,Solid work van no rust. Built in shelves.Runs ...,V6 motor auto transmission shifts smooth . V6...,Buyer has been in van for more than 30 months...
2,2,"$3,000",Jefferson,2007 Jeep Patriot,https://boone.craigslist.org/cto/d/jefferson-2...,Nov 20,"I have a 2007 Jeep Patriot clean title, 4x4, 2...","2007 Jeep Patriot clean title, 4x4, 2.4L engi...","2007 Jeep Patriot clean title, 4x4, 2.4L engi..."
3,3,"$3,500",NaN,2004 Mitsubishi Lancer Sportback LS,https://boone.craigslist.org/cto/d/jefferson-2...,Nov 19,Rare 2004 Mitsubishi Lancer Sportback LS\n\nTh...,The only year the Sportback was produced in t...,The 2004 Mitsubishi Lancer Sportback LS LS is...
4,4,"$6,999",Gastonia,2003 Honda Element EX AWD,https://boone.craigslist.org/ctd/d/mc-adenvill...,Nov 18,**ONE OWNER VEHICLE**2003 Honda Element EX AWD...,2003 Honda Element EX AWD 4dr SUV Automatic 2...,2003 Honda Element EX AWD 4dr SUV Automatic 2...


In [ ]:
calc_rouge_scores(df2, 'Description', 'tt_result','desc_tt_result_rouge')
calc_rouge_scores(df2, 'Description', 'fine_tt_result','desc_fine_tt_result_rouge')
df2.head()

,Unnamed: 0,Price,Location,Title,Link,Date,Description,tt_result,fine_tt_result,desc_tt_result_rouge,desc_fine_tt_result_rouge
0,0,"$19,990",TOUCHLESS DELIVERY TO YOUR HOME,2010 Toyota RAV4 Limited Sport Utility 4D suv ...,https://boone.craigslist.org/ctd/d/boone-2010-...,Nov 23,"At Carvana, we go miles beyond the extra mile....","Carvana provides a convenient, fast, and hass...",Carvana stock# 2001393850 is 2001393850 . Tex...,"{'rouge-1': {'r': 0.9310344827586207, 'p': 0.0...","{'rouge-1': {'r': 0.8, 'p': 0.0314960629921259..."
1,1,"$1,500",Newland NC,2003 ford e150 work van,https://boone.craigslist.org/cto/d/montezuma-2...,Nov 22,Solid work van no rust. Built in shelves.Runs ...,V6 motor auto transmission shifts smooth . V6...,3-year year old van driven by a work van with...,"{'rouge-1': {'r': 1.0, 'p': 0.3684210526315789...","{'rouge-1': {'r': 0.4, 'p': 0.2105263157894736..."
2,2,"$3,000",Jefferson,2007 Jeep Patriot,https://boone.craigslist.org/cto/d/jefferson-2...,Nov 20,"I have a 2007 Jeep Patriot clean title, 4x4, 2...","2007 Jeep Patriot clean title, 4x4, 2.4L engi...","Jeep Patriot clean title, 4x4, 2.4L engine, a...","{'rouge-1': {'r': 0.9473684210526315, 'p': 0.2...","{'rouge-1': {'r': 0.8709677419354839, 'p': 0.4..."
3,3,"$3,500",NaN,2004 Mitsubishi Lancer Sportback LS,https://boone.craigslist.org/cto/d/jefferson-2...,Nov 19,Rare 2004 Mitsubishi Lancer Sportback LS\n\nTh...,The only year the Sportback was produced in t...,Rare 2004 Mitsubishi Lancer Sportback LS is o...,"{'rouge-1': {'r': 0.9523809523809523, 'p': 0.1...","{'rouge-1': {'r': 0.8285714285714286, 'p': 0.2..."
4,4,"$6,999",Gastonia,2003 Honda Element EX AWD,https://boone.craigslist.org/ctd/d/mc-adenvill...,Nov 18,**ONE OWNER VEHICLE**2003 Honda Element EX AWD...,2003 Honda Element EX AWD 4dr SUV Automatic 2...,2003 Honda Element EX AWD 4dr SUV Automatic 2...,"{'rouge-1': {'r': 0.9285714285714286, 'p': 0.2...","{'rouge-1': {'r': 0.8214285714285714, 'p': 0.2..."


In [ ]:
%%time
get_t5_result(df2)

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


CPU times: user 10.2 s, sys: 82 ms, total: 10.3 s
Wall time: 10.3 s


In [ ]:
calc_rouge_scores(df2, 'fine_tt_result','t5_result','fine_tt_result_t5_result_rouge')
calc_rouge_scores(df2, 'Description','t5_result','desc_t5_result_rouge')
df2.head()

,Unnamed: 0,Price,Location,Title,Link,Date,Description,tt_result,fine_tt_result,t5_result,fine_tt_result_t5_result_rouge,desc_t5_result_rouge
0,0,"$19,990",TOUCHLESS DELIVERY TO YOUR HOME,2010 Toyota RAV4 Limited Sport Utility 4D suv ...,https://boone.craigslist.org/ctd/d/boone-2010-...,Nov 23,"At Carvana, we go miles beyond the extra mile....","Carvana provides a convenient, fast, and hass...",Carvana is a leading manufacturer that helps ...,"carvana offers a convenient, fast, and hassle-...","{'rouge-1': {'r': 0.6060606060606061, 'p': 0.3...","{'rouge-1': {'r': 0.9090909090909091, 'p': 0.0..."
1,1,"$1,500",Newland NC,2003 ford e150 work van,https://boone.craigslist.org/cto/d/montezuma-2...,Nov 22,Solid work van no rust. Built in shelves.Runs ...,V6 motor auto transmission shifts smooth . V6...,Buyer has been in van for more than 30 months...,a solid work van no rust. built in shelves.Run...,"{'rouge-1': {'r': 0.38461538461538464, 'p': 0....","{'rouge-1': {'r': 0.7692307692307693, 'p': 0.2..."
2,2,"$3,000",Jefferson,2007 Jeep Patriot,https://boone.craigslist.org/cto/d/jefferson-2...,Nov 20,"I have a 2007 Jeep Patriot clean title, 4x4, 2...","2007 Jeep Patriot clean title, 4x4, 2.4L engi...","2007 Jeep Patriot clean title, 4x4, 2.4L engi...","i have a 2007 Jeep Patriot clean title, 4x4, 2...","{'rouge-1': {'r': 0.7096774193548387, 'p': 0.5...","{'rouge-1': {'r': 0.967741935483871, 'p': 0.47..."
3,3,"$3,500",NaN,2004 Mitsubishi Lancer Sportback LS,https://boone.craigslist.org/cto/d/jefferson-2...,Nov 19,Rare 2004 Mitsubishi Lancer Sportback LS\n\nTh...,The only year the Sportback was produced in t...,The 2004 Mitsubishi Lancer Sportback LS LS is...,the only year the Lancer Sportback was produce...,"{'rouge-1': {'r': 0.47058823529411764, 'p': 0....","{'rouge-1': {'r': 1.0, 'p': 0.3366336633663366..."
4,4,"$6,999",Gastonia,2003 Honda Element EX AWD,https://boone.craigslist.org/ctd/d/mc-adenvill...,Nov 18,**ONE OWNER VEHICLE**2003 Honda Element EX AWD...,2003 Honda Element EX AWD 4dr SUV Automatic 2...,2003 Honda Element EX AWD 4dr SUV Automatic 2...,Honda Element EX AWD 4dr SUV Automatic 2.4L I4 .,"{'rouge-1': {'r': 1.0, 'p': 0.2222222222222222...","{'rouge-1': {'r': 1.0, 'p': 0.1111111111111111..."


In [ ]:
df2_avg_desc_tt_result_rouge = sum_rpf(df1,'desc_tt_result_rouge')
df2_avg_desc_fine_tt_result_rouge = sum_rpf(df1,'desc_fine_tt_result_rouge')
df2_avg_desc_t5_result_rouge = sum_rpf(df1,'desc_t5_result_rouge')
df2_avg_fine_tt_result_t5_result_rouge = sum_rpf(df1,'fine_tt_result_t5_result_rouge')

In [ ]:
print('df2_avg_desc_tt_result_rouge \n\n',df2_avg_desc_tt_result_rouge)
print('\ndf2_avg_desc_fine_tt_result_rouge \n\n',df2_avg_desc_fine_tt_result_rouge)
print('\ndf2_avg_desc_t5_result_rouge \n\n',df2_avg_desc_t5_result_rouge)
print('\ndf2_avg_fine_tt_result_t5_result_rouge \n\n',df2_avg_fine_tt_result_t5_result_rouge)

df2_avg_desc_tt_result_rouge 

     rouge-1   rouge-2   rouge-l
r  0.833824  0.678088  0.829514
p  0.281056  0.235096  0.279894
f  0.374923  0.304921  0.373203

df2_avg_desc_fine_tt_result_rouge 

     rouge-1   rouge-2   rouge-l
r  0.739313  0.561626  0.729449
p  0.279364  0.219641  0.275793
f  0.363455  0.276205  0.358765

df2_avg_desc_t5_result_rouge 

     rouge-1   rouge-2   rouge-l
r  0.838299  0.702175  0.832013
p  0.293415  0.245315  0.290644
f  0.388536  0.319541  0.385247

df2_avg_fine_tt_result_t5_result_rouge 

     rouge-1   rouge-2   rouge-l
r  0.430997  0.315174  0.424998
p  0.384899  0.267475  0.379449
f  0.387372  0.274020  0.381856
